In [1]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import cv2
import numpy as np
import densenet as dn
import pdb
# used for logging to TensorBoard
from tensorboard_logger import configure, log_value

In [2]:
#model = dn.DenseNet3(100, 4, 24, reduction=args.reduce,bottleneck=args.bottleneck, dropRate=args.droprate, small_inputs = False)
parser = argparse.ArgumentParser(description='PyTorch DenseNet Training')
parser.add_argument('--epochs', default=510, type=int,
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=500, type=int,
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=16, type=int,
                    help='mini-batch size (default: 32)')
parser.add_argument('--lr', '--learning-rate', default=0.01, type=float,
                    help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    help='weight decay (default: 1e-4)')
parser.add_argument('--print-freq', '-p', default=10, type=int,
                    help='print frequency (default: 10)')
parser.add_argument('--layers', default=100, type=int,
                    help='total number of layers (default: 100)')
parser.add_argument('--growth', default=12, type=int,
                    help='number of new channels per layer (default: 12)')
parser.add_argument('--droprate', default=0, type=float,
                    help='dropout probability (default: 0.0)')
parser.add_argument('--no-augment', dest='augment', action='store_false',
                    help='whether to use standard augmentation (default: False)')
parser.add_argument('--reduce', default=0.5, type=float,
                    help='compression rate in transition stage (default: 0.5)')
parser.add_argument('--no-bottleneck', dest='bottleneck', action='store_false',
                    help='To not use bottleneck block')
parser.add_argument('--resume', default='C:\\Users\\fs\\Desktop\\densenet-pytorch-master\\runs_k=12\\DenseNet_Unet_fs\\model_best.pth', type=str,
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--name', default='DenseNet_Unet_fs', type=str,
                    help='name of experiment')
parser.add_argument('--tensorboard',
                    help='Log progress to TensorBoard', action='store_true')
parser.set_defaults(bottleneck=True)
parser.set_defaults(augment=False)
global args, best_prec1
args = parser.parse_args([])

In [41]:
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])

transform_train = transforms.Compose([
            normalize,
            ])


In [25]:
model = dn.DenseNet3(100, 4, 12, bottleneck=args.bottleneck,  small_inputs = False)
model = model.cuda()
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}' (epoch {})"
                .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

=> loading checkpoint 'C:\Users\fs\Desktop\densenet-pytorch-master\runs_k=12\DenseNet_Unet_fs\model_best.pth'
=> loaded checkpoint 'C:\Users\fs\Desktop\densenet-pytorch-master\runs_k=12\DenseNet_Unet_fs\model_best.pth' (epoch 296)


In [46]:
#img = cv2.imread('flaw.jpg')
img = cv2.imread('bubble(9).bmp').astype(np.float32)
img.shape
img = np.transpose(img, axes=[2, 0, 1])

#img = np.expand_dims(img, axis=0)
img.shape

#type(input)
input = torch.from_numpy(img)
input = transform_train(input)

In [47]:

input = input.unsqueeze(0)
input = input.cuda()
input = torch.autograd.Variable(input)
input.shape

model.eval()
output = model(input)
print(output)
x= torch.max(output,1)
classes = []
with open('classes.txt', 'r') as list_:
    for line in list_:
        classes.append(line.rstrip('\n'))

print('Prediction: ', str(classes[x[1]]))

tensor([[-5082.2334,  6692.0347, -3067.8418,  1459.7487]], device='cuda:0',
       grad_fn=<AddmmBackward>)
Prediction:  flaw


In [21]:
topk=(1,)
maxk = max(topk)
_, pred = output.topk(maxk, 1, True, True)

In [23]:
pred = pred.t()

In [24]:
pred


tensor([[1]], device='cuda:0')